In [10]:
import requests

import numpy as np
import pandas as pd

import json

import math

In [11]:
#funder = 'US'
funder = 'bmgf'

projects_url = f'http://127.0.0.1:5000/graphql?query=query%20fundersList%20%7B%0A%09funder(code%3A%20%22{funder}%22)%20%7B%0A%20%20%20%20projects%20%7B%0A%20%20%20%20%20%20projectId%0A%20%20%20%20%20%20projectName%0A%20%20%20%20%20%20funderRef%0A%20%20%20%20%20%20donorSector%0A%20%20%20%20%20%20donorCode%0A%20%20%20%20%20%20donorName%0A%20%20%20%20%20%20recipientCountry%0A%20%20%20%20%20%20recipientSector%0A%20%20%20%20%20%20recipientName%0A%20%20%20%20%20%20totalCommitted%0A%20%20%20%20%20%20totalSpent%0A%20%20%20%20%20%20totalCurrency%0A%20%20%20%20%20%20source%0A%20%20%20%20%20%20donorAmountUnspec%0A%20%20%20%20%20%20recipientAmountUnspec%0A%20%20%20%20%7D%0A%20%20%7D%20%0A%7D&operationName=fundersList'

projects_json = requests.get(projects_url).json()['data']['funder']['projects']

In [12]:
locations = pd.read_csv('country_locations_and_areas.csv')
locations.set_index('name', inplace=True)
locations.loc[:, 'area'] = locations.loc[:, 'area'].astype(float)

In [13]:
def radius(area):
    return math.sqrt(area / math.pi)

def lat_extent(radius):
    return (radius / 111)

# 111,111 * cos(latitude) meters in the x direction is 1 degree
# 111111 * cos(lat) / radius = degrees
def lon_extent(radius, latitude):
    return (radius / abs(111 * math.cos(latitude)) )

In [14]:
162.423900 / abs(111111 * math.cos(23.424076))

0.010636610766002396

In [15]:
# r = sqrt(area / pi)
locations.loc[:, 'radius'] = locations.loc[:, 'area'].map(radius)

locations.loc[:, 'lat_extent'] = locations.loc[:, 'radius'].map(lat_extent)
locations.loc[:, 'lon_extent'] = np.vectorize(lon_extent, otypes=["O"]) (locations['radius'], locations['latitude'])

locations.head(10)

,country,latitude,longitude,area,radius,lat_extent,lon_extent
name,,,,,,,
Andorra,AD,42.546245,1.601554,468.0,12.205287,0.109958,0.818521
United Arab Emirates,AE,23.424076,53.847818,82880.0,162.423900,1.463278,10.6472
Afghanistan,AF,33.939110,67.709953,647500.0,453.988603,4.089987,5.01961
Antigua and Barbuda,AG,17.060816,-61.796428,442.0,11.861407,0.106860,0.494211
Albania,AL,41.153332,20.168331,28748.0,95.659671,0.861799,0.905699
Armenia,AM,40.069099,45.038189,29800.0,97.394223,0.877425,1.2241
Netherlands Antilles,AN,12.226079,-69.060087,960.0,17.480775,0.157484,0.167064
Angola,AO,-11.202692,17.873887,1246700.0,629.949947,5.675225,27.5979
Argentina,AR,-38.416097,-63.616672,2780400.0,940.759697,8.475313,11.2436


In [16]:
locations.loc[(locations.country == 'US')]

,country,latitude,longitude,area,radius,lat_extent,lon_extent
name,,,,,,,
United States of America,US,37.09024,-95.712891,9525067.0,1741.241796,15.686863,19.1235


In [17]:
us_from = {
    'name':	'United States of America',
    'coordinates': [-95.712891, 37.09024],
    'extent':  [19.1235, 15.686863],
}

gates_from = {
    'name':	'Bill & Melinda Gates Foundation',
    'coordinates': [-122.3321, 47.6062],
    'extent':  [1, 1],
}

In [18]:
new_proj_list = []

max_outbound = 0

for project in projects_json:
    
    proj = {}
    
    try:
        proj['from'] = gates_from

        from_coords = [locations.loc[project['recipientName'], 'longitude'],
                       locations.loc[project['recipientName'], 'latitude']]

        from_extent = [locations.loc[project['recipientName'], 'lon_extent'],
                       locations.loc[project['recipientName'], 'lat_extent']] 


        to_coords = [locations.loc[project['recipientName'], 'longitude'],
                     locations.loc[project['recipientName'], 'latitude']]

        to_extent = [locations.loc[project['recipientName'], 'lon_extent'],
                     locations.loc[project['recipientName'], 'lat_extent']]
        
        proj['to'] = {'name': project['recipientName'], 'coordinates': to_coords, 'extent': to_extent}

        proj['name'] = project['projectName']
        proj['outbound'] = project['totalCommitted']
        
        if proj['outbound'] > max_outbound:
            max_outbound = proj['outbound']
        
        # print(proj)
        
        new_proj_list.append(proj)
    except KeyError:
        pass

In [19]:
max_outbound

77601500.0

In [20]:
new_proj_list[0]

{'from': {'name': 'Bill & Melinda Gates Foundation',
  'coordinates': [-122.3321, 47.6062],
  'extent': [1, 1]},
 'to': {'name': 'Zambia',
  'coordinates': [27.849332, -13.133897],
  'extent': [5.229263345450484, 4.409488671967828]},
 'name': 'ACTION Global Health Partnership Advocacy Anchor Grant 2014-2017',
 'outbound': 599180.9689999999}

In [21]:
with open('Gates_funding_data.json', 'w') as outfile:  
    json.dump(new_proj_list, outfile)